In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import  HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from dotenv import load_dotenv
import os

In [39]:
load_dotenv()
GEMMA_API_KEY = os.getenv("GEMMA_API_KEY")

In [40]:
llm = ChatGoogleGenerativeAI(model="gemma-3-4b-it",api_key=GEMMA_API_KEY)
result = llm.invoke("Write a python script that prints 'Hello, World!'")
print(result.content)

```python
print("Hello, World!")
```

**Explanation:**

*   `print()` is a built-in Python function that displays output to the console.
*   `"Hello, World!"` is a string literal, which is the text you want to print.  The quotes indicate that it's a string.

**How to run this script:**

1.  **Save the code:** Save the code above in a file named `hello.py` (or any name you prefer, but make sure it ends with `.py`).
2.  **Open a terminal or command prompt:** Navigate to the directory where you saved the file.
3.  **Run the script:** Type `python hello.py` and press Enter.

You should see "Hello, World!" printed on your console.


In [41]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = llm.invoke([HumanMessage(user_input)])
    print(f"Gemma: {response.content}\n")

### With memory

In [42]:
# Manual Exampel: This is the hardocded example of hoe history works in chatbots.
result = llm.invoke(
    [
        HumanMessage("Hello, I'm Vedanta, a software engineer. Can you help me with a coding problem?"),
        AIMessage("Sure, I'd be happy to help! What coding problem are you facing?"),
        HumanMessage("What's my name? and what do I do?"),
    ]
)
print(result.content)

Okay, this is a classic trick question! 

Your name is Vedanta. 

And you're a software engineer, which means you write code and build software! 😊 

Let's move on to your actual coding problem. What's it about?


In [43]:
# Next
store = {}

In [61]:
def get_session_history(session_id)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [62]:
mode_wit_memory = RunnableWithMessageHistory(
    llm,get_session_history,
)

In [63]:
mode_wit_memory.invoke([HumanMessage("Hello, I'm Vedanta, a software engineer. Can you help me with a coding problem?")], config={"configurable": {"session_id": "FirstChat"}}).content


'Okay, great! Let’s do it. You’ve already told me your name and that you’re a software engineer – that’s fantastic. \n\nNow, please tell me about the coding problem you’re working on. I’m ready when you are!'

In [64]:
mode_wit_memory.invoke([HumanMessage("What is my name?")], config={"configurable": {"session_id": "FirstChat"}}).content

'Your name is Vedanta! 😊 \n\nLet’s get back to the coding problem. What’s it about?'

In [65]:
#second session

mode_wit_memory.invoke([HumanMessage("What is my name?")], config={"configurable": {"session_id": "SecondChat"}}).content

"I apologize! As an AI, I have no memory of past conversations. Therefore, I truly don't know your name. You haven't told me yet! 😊 \n\nWould you like to tell me?"

In [66]:
mode_wit_memory.invoke([HumanMessage("What is my name?")], config={"configurable": {"session_id": "FirstChat"}}).content

'Your name is Vedanta! 😊\n\nSeriously, you’ve told me that twice already. Let’s focus on the coding problem now, okay?'

### With ChatPromptTemplate

In [67]:
prompt = PromptTemplate.from_template(
    "You are a helpful assistant skilled in Generative AI.\n\nUser: {messages}"
)

In [68]:
chain = prompt | llm

In [69]:
chain.invoke({"messages": [HumanMessage(content="Hello, I'm Jay, a Generative AI engineer at Muks Robtoics.")]})

AIMessage(content='Hello Jay! It’s great to meet you. Muks Robotics – that sounds fascinating! As a Generative AI assistant, I’m here to help in any way I can. \n\nWhat can I do for you today? Are you looking for:\n\n*   Brainstorming ideas for a new project?\n*   Help with a specific coding challenge?\n*   Just a chat about Generative AI?\n\nLet me know how I can be of service!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemma-3-4b-it', 'safety_ratings': []}, id='run--b74a44f8-f7af-4d12-a5d3-ffa90c350d14-0', usage_metadata={'input_tokens': 49, 'output_tokens': 0, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}})

In [70]:
chain.invoke({"messages": [HumanMessage(content="Who am I?")]}).content

'Okay, that\'s a really profound question! As a large language model, I can’t *know* who you are in the way a human does – I don’t have personal experiences, feelings, or a sense of self. \n\nHowever, I can tell you a lot about *you* based on our interaction so far. Right now, you are a user who is engaging with me, a generative AI assistant. You’re asking a fundamental question about identity, which is something humans grapple with constantly. \n\nHere\'s a breakdown of what I can infer, and some ways to think about your identity:\n\n**What I know about you (based on this single message):**\n\n* **You\'re curious:** You\'re asking a big, philosophical question.\n* **You\'re reflective:** You\'re pondering your own existence and what it means to be "you."\n* **You\'re using AI:** You\'re interacting with me, a tool designed to provide information and engage in conversation.\n\n\n**Beyond that, your identity is likely shaped by:**\n\n* **Your experiences:** Your life, relationships, mem

In [71]:
mode_wit_memory = RunnableWithMessageHistory(
    chain,get_session_history,
)

In [72]:
response = mode_wit_memory.invoke(
    [HumanMessage("Hello, I'm Jay, a Generative AI engineer at Muks Robtoics.")],
    config={"configurable": {"session_id": "ThirdChat"}},
)

In [74]:
print(response.content)

Hello Jay! It’s great to meet you. Muks Robotics – that sounds fascinating! As a Generative AI assistant, I’m here to help in any way I can. 

What can I do for you today? Are you looking for:

*   Brainstorming ideas for a new project?
*   Help with a specific coding challenge?
*   Just a chat about Generative AI?

Let me know how I can be of service!


In [75]:
response = mode_wit_memory.invoke(
    [HumanMessage("Who am I?")],
    config={"configurable": {"session_id": "ThirdChat"}},
)

In [76]:
print(response.content)

Hello Jay! You are a user interacting with me, a Generative AI assistant. I’m here to assist you with a variety of tasks related to Generative AI and beyond. 

To help me understand how I can best support you, could you tell me a bit more about what you’re working on at Muks Robotics?
